In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [22]:
url_inicial = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive'
url_root = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive'
r=requests.get(url_inicial)

In [23]:
soup = BeautifulSoup(r.text, 'html.parser')

In [24]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', id='issue')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/30',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/29',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/28',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/27',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/26',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/25',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/24',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/23',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/21',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/22',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/20',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/g

In [25]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [26]:
vol2

['http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1011',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1012',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1013',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1014',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1015',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1016',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1017',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1018',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1019',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/1020',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/article/view/958',
 'http://www.gestionyp

In [27]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [28]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive


In [29]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

152

In [30]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [31]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Gestión y Política Pública'
    a='V. Ciencias Sociales'
    tem='Administración Pública y Estudio de las Organizaciones'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [32]:
list_scraper=list_scraper[0:152]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [33]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,¿Qué aporta la escucha al capital social?: Lec...,Los estudios de administración pública afirman...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
1,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Análisis de la contribución del gasto público ...,El presente artículo tiene por objetivo analiz...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
2,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Instrumentos de financiación del desarrollo ur...,En el marco de las transformaciones urbanas y ...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
3,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,La acción de los expertos en contexto: La apli...,Frente a la complejidad técnica requerida para...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
4,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Vulnerabilidad a la pobreza y juventud: Entre ...,Este artículo revisa los factores de riesgo y ...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
...,...,...,...,...,...,...,...,...
147,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Una revisión histórica del desarrollo del gobi...,. El objetivo de este trabajo es hacer una rev...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
148,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,La institucionalización del análisis de políti...,. Este artículo se centra en la institucionali...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
149,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Un Estado más efectivo: Capacidades para el di...,Reseña de Mauricio I. Dussauge Laguna al libro...,None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...
150,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Leveling the Playing Field: Transnational Regu...,"Reseña de Enrique García Martínez al libro ""Le...",None,http://www.gestionypoliticapublica.cide.edu/oj...,http://www.gestionypoliticapublica.cide.edu/oj...


In [34]:
df_catalogo.to_csv('Revista065.01.csv', index=False)